In [4]:
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
#from transformers import pipeline
# For fine-tuning
import torch
#from torch import nn
#from torch.utils.data import Dataset, DataLoader
# For processing
import numpy as np
import pandas as pd
import os
import json

In [7]:
folder_path = "./dreams"
count = 0
all_files = []
for i in os.listdir(folder_path):
    if i.endswith('.json'):
        count+=1
        all_files.append(i)
dreams = []
for i in all_files:
    full_name  = os.path.join(folder_path,i)
    with open(full_name,'r') as f:
        data = json.load(f)
        for x in data['dreams']:
            dreams.append({
                "content":x.get("content","")
            })
              
df = pd.DataFrame(dreams)
#print(count)
df.size

FileNotFoundError: [Errno 2] No such file or directory: './dreams'

In [5]:
from transformers import BertModel, BertTokenizer
import torch

# 1. Load pre-trained BERT model and tokenizer
model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 2. Prepare input text
text = "I had a dream about flying"

# 3. Tokenize and encode the text
inputs = tokenizer(
    text, 
    return_tensors='pt',  # Return PyTorch tensors
    max_length=512,        # Maximum sequence length
    truncation=True,       # Truncate longer sequences
    padding=True           # Pad shorter sequences
)

# 4. Get BERT embeddings
with torch.no_grad():
    outputs = model(**inputs)

# 5. Extract different types of embeddings
last_hidden_state = outputs.last_hidden_state  # Token-level embeddings
pooled_output = outputs.pooler_output  # Sentence-level representation

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [6]:
last_hidden_state

tensor([[[ 0.1249,  0.2264, -0.3519,  ..., -0.2224,  0.2284,  0.1683],
         [ 0.0719, -0.3525, -0.1092,  ..., -0.5893,  0.4822, -0.1437],
         [ 0.1657, -0.1575,  0.3219,  ..., -0.6984, -0.1438, -0.2681],
         ...,
         [ 0.1226,  0.6299, -0.2976,  ..., -0.4775, -0.0297, -0.0102],
         [ 0.0624, -0.0127, -0.6580,  ...,  0.2324,  0.4505, -0.2222],
         [ 0.7634,  0.2097, -0.3422,  ..., -0.0912, -0.5637, -0.3376]]])